In [1]:
### no brand names and search terms vector

In [1]:
%load_ext Cython

In [2]:
%%time
%%cython
from gensim.models import doc2vec
import nltk
import re
def split_sentence (sentence):
    return re.split('\W+',sentence)
    
class MyDocs(object):
    def __iter__(self):
        for i, text in enumerate(open("txt_pd_st.txt")): # doesn't use brand name
            yield doc2vec.LabeledSentence(words=split_sentence(text), tags=['%s' % i])
            # Train the doc2vec model
cdef desp = MyDocs()
model = doc2vec.Doc2Vec(desp, size = 200, window = 8, min_count = 5, workers = 4)
model.save('distance_pd_st.model')

CPU times: user 10min 50s, sys: 20.3 s, total: 11min 11s
Wall time: 4min 28s


In [3]:
model = doc2vec.Doc2Vec.load('distance_pd_st.model')

In [4]:
# Calculate the vector of search term according to our model
#def input_vec(x):
#    model.infer_vector(split_sentence(x))
# map(input_vec,open("txt_st.txt))
import numpy as np
pt_arrays = np.zeros((240760, 200))
for i,text in enumerate(open("txt_dsp.txt")): # input search terms 
    pt_arrays[i]=model.infer_vector(split_sentence(text))   
pt_arrays

array([[-0.02813668, -0.09714227, -0.08055815, ...,  0.00760074,
        -0.0365501 , -0.00546911],
       [-0.02692451,  0.04056466, -0.13432434, ...,  0.0394102 ,
         0.01681765, -0.10102455],
       [ 0.10535594,  0.03914834, -0.15584618, ...,  0.0040602 ,
        -0.0205088 ,  0.06099103],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [5]:
import numpy as np
st_arrays = np.zeros((240760, 200))
for i,text in enumerate(open("txt_st.txt")): # input search terms 
    st_arrays[i]=model.infer_vector(split_sentence(text))   
st_arrays

array([[  2.30096858e-02,  -3.25129703e-02,  -4.89307977e-02, ...,
         -4.66523804e-02,   2.09233519e-02,  -1.38169178e-03],
       [ -8.30502715e-03,  -5.22310939e-03,   2.84305797e-03, ...,
         -1.38077391e-02,  -2.75015533e-02,  -1.61226766e-04],
       [ -6.94953278e-03,  -3.08607295e-02,   1.99920051e-02, ...,
          5.00306999e-03,   5.85368508e-03,   6.03485592e-02],
       ..., 
       [  3.27529781e-03,   3.11468951e-02,  -2.87304502e-02, ...,
          9.27566644e-03,   6.97412994e-04,   1.29626784e-02],
       [ -2.46330481e-02,   1.02390964e-02,  -1.34011619e-02, ...,
          2.60553192e-02,   5.02078421e-02,   2.06296500e-02],
       [ -1.55564491e-02,  -8.44577677e-04,  -5.46263473e-05, ...,
         -4.39195521e-03,   1.39905503e-02,  -2.30611768e-03]])

In [6]:
## calculate the cosin distance between 2 vectors to generate the 3rd features
# Compute the cosine similarity values between the input text and all archived reviews
# cossims_with_input = map(lambda v: cossim(input_vec, v), model.docvecs) 
# need to chaneg the code into rows: calculate the cosin distance between the vectors in same rows
import numpy as np
import pandas as pd
# Calculate the cosine similarity between two vecotrs 
def cossim(v1, v2):
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2))

cossims_with_st=np.zeros((240760,1))
for i in range(240760):
    cossims_with_st[i] = cossim(st_arrays[i], pt_arrays[i])

In [7]:
np.savetxt("distance_pd_st.csv",cossims_with_st, delimiter=",")

In [8]:
coswithst=pd.read_csv("distance_pd_st.csv",header=None)
coswithst


,0
0,0.119862
1,-0.052897
2,0.071725
3,0.025093
4,0.048474
5,0.193320
6,0.022498
7,0.073914
8,-0.081933
9,0.027945


In [ ]:
cws_trn=coswithst.iloc[:num_train]
cws_tst=coswithst.iloc[num_train:]